### 9 pebbles
One pebble in center: 8 pebbles for each corner

In [ ]:
# Nine 3cm pebbles
# Side-length: 2*3.4641016151

##########
# import #
##########
import gmsh
import sys

######################
# Create Sphere Mesh #
######################
# Initialize the gmsh API
gmsh.initialize()
# Create a new model "9pebs"
gmsh.model.add("9pebs")

# Define radius of sphere and side length of bounding box
sphere_radius = 3.0 #cm
half_length = 3.5921476745623537 # Updated for PF = 0.61 #3.4641016151 # cm
side_length = half_length*2

# Creates a sphere centered at (0,0,0) with given radius
p_1 = gmsh.model.occ.addSphere(-half_length,  half_length,  half_length, sphere_radius)
p_2 = gmsh.model.occ.addSphere( half_length,  half_length,  half_length, sphere_radius)
p_3 = gmsh.model.occ.addSphere( half_length, -half_length,  half_length, sphere_radius)
p_4 = gmsh.model.occ.addSphere(-half_length, -half_length,  half_length, sphere_radius)
p_5 = gmsh.model.occ.addSphere( 0,            0,            0,           sphere_radius)
p_6 = gmsh.model.occ.addSphere(-half_length,  half_length, -half_length, sphere_radius)
p_7 = gmsh.model.occ.addSphere( half_length,  half_length, -half_length, sphere_radius)
p_8 = gmsh.model.occ.addSphere( half_length, -half_length, -half_length, sphere_radius)
p_9 = gmsh.model.occ.addSphere(-half_length, -half_length, -half_length, sphere_radius)

# creates the bounding box.
bound_box = gmsh.model.occ.addBox(-half_length, -half_length, -half_length, side_length, side_length, side_length) # addBox(x, y, z, dx, dy, dz, tag=-1)


# gmsh.model.occ.cut(objectDimTags, toolDimTags, tag=-1, removeObject=True, removeTool=True)
# The function "cut" subtracts one volume from another volume.
# In this case, we are cutting the pebble sphere from the bounding box.
# objectDimTags: The entities to be cut. Each pair (dimension, tag) specifies an object
# toolDimTags: The cutting tools that remove material from objectDimTags.
# removeTool: boolean -> if set to True, removes the cutting tools: We don't want this because we want to mesh the pebble sphere also.
intersect_peb_1 = gmsh.model.occ.intersect([(3, p_1)], [(3, bound_box)], removeObject=True, removeTool=False)
cut_box_1       = gmsh.model.occ.cut([(3, bound_box)], [(3, p_1)], removeTool=False)

intersect_peb_2 = gmsh.model.occ.intersect([(3, p_2)], [(3, bound_box)], removeObject=True, removeTool=False)
cut_box_2       = gmsh.model.occ.cut([(3, bound_box)], [(3, p_2)], removeTool=False)

intersect_peb_3 = gmsh.model.occ.intersect([(3, p_3)], [(3, bound_box)], removeObject=True, removeTool=False)
cut_box_3       = gmsh.model.occ.cut([(3, bound_box)], [(3, p_3)], removeTool=False)

intersect_peb_4 = gmsh.model.occ.intersect([(3, p_4)], [(3, bound_box)], removeObject=True, removeTool=False)
cut_box_4       = gmsh.model.occ.cut([(3, bound_box)], [(3, p_4)], removeTool=False)

intersect_peb_5 = gmsh.model.occ.intersect([(3, p_5)], [(3, bound_box)], removeObject=True, removeTool=False)
cut_box_5       = gmsh.model.occ.cut([(3, bound_box)], [(3, p_5)], removeTool=False)

intersect_peb_6 = gmsh.model.occ.intersect([(3, p_6)], [(3, bound_box)], removeObject=True, removeTool=False)
cut_box_6       = gmsh.model.occ.cut([(3, bound_box)], [(3, p_6)], removeTool=False)

intersect_peb_7 = gmsh.model.occ.intersect([(3, p_7)], [(3, bound_box)], removeObject=True, removeTool=False)
cut_box_7       = gmsh.model.occ.cut([(3, bound_box)], [(3, p_7)], removeTool=False)

intersect_peb_8 = gmsh.model.occ.intersect([(3, p_8)], [(3, bound_box)], removeObject=True, removeTool=False)
cut_box_8       = gmsh.model.occ.cut([(3, bound_box)], [(3, p_8)], removeTool=False)

intersect_peb_9 = gmsh.model.occ.intersect([(3, p_9)], [(3, bound_box)], removeObject=True, removeTool=False)
cut_box_9       = gmsh.model.occ.cut([(3, bound_box)], [(3, p_9)], removeTool=False)



# Synchronize the CAD kernel with the Gmsh model
# ChatGPT says: 
#    "When working with OpenCASCADE(OCC)-based functions (gmsh.model.occ.*), changes happen only within the OCC kernel, 
#    and Gmsh doesn't immediately "see" them. Calling gmsh.model.occ.synchronize() commits these changes to the Gmsh model."
gmsh.model.occ.synchronize()

# Define Surfaces!
# Get all 2D surfaces (faces)
surfaces = gmsh.model.getEntities(2)
# Determine the bounding surfaces of the cube based on location
tol = 1e-6
xmin = ymin = zmin = -half_length - tol
xmax = ymax = zmax =  half_length + tol

outer_surfaces = []

for (dim, tag) in surfaces:
    com = gmsh.model.occ.getCenterOfMass(dim, tag)
    x, y, z = com
    if (abs(x - xmin) < tol or abs(x - xmax) < tol or
        abs(y - ymin) < tol or abs(y - ymax) < tol or
        abs(z - zmin) < tol or abs(z - zmax) < tol):
        outer_surfaces.append(tag)

# Create a single physical group for all outer surfaces
gmsh.model.addPhysicalGroup(2, outer_surfaces, tag=100)
gmsh.model.setPhysicalName(2, 100, "outer_boundary")


# Identify the volumes: we expect two volumes
volEntities = gmsh.model.getEntities(3) # dim=3 specifies that we're looking for volumes (not polygons I suppose)
print("Volumes:", volEntities)

# Define physical groups
gmsh.model.addPhysicalGroup(3, [p_1],       tag=1) # We specify "physical groups" (I think) to identify regions where we can assign properties to regions (i.e. Multi-group cross sections)
gmsh.model.setPhysicalName(3, 1, "p_1")            # An integer ID is not enough, we want to assign a name to this region.
gmsh.model.addPhysicalGroup(3, [p_2],       tag=2) 
gmsh.model.setPhysicalName(3, 2, "p_2")                
gmsh.model.addPhysicalGroup(3, [p_3],       tag=3) 
gmsh.model.setPhysicalName(3, 3, "p_3")               
gmsh.model.addPhysicalGroup(3, [p_4],       tag=4) 
gmsh.model.setPhysicalName(3, 4, "p_4")               
gmsh.model.addPhysicalGroup(3, [p_5],       tag=5) 
gmsh.model.setPhysicalName(3, 5, "p_5")               
gmsh.model.addPhysicalGroup(3, [p_6],       tag=6) 
gmsh.model.setPhysicalName(3, 6, "p_6")               
gmsh.model.addPhysicalGroup(3, [p_7],       tag=7) 
gmsh.model.setPhysicalName(3, 7, "p_7")               
gmsh.model.addPhysicalGroup(3, [p_8],       tag=8) 
gmsh.model.setPhysicalName(3, 8, "p_8")               
gmsh.model.addPhysicalGroup(3, [p_9],       tag=9) 
gmsh.model.setPhysicalName(3, 9, "p_9")               
gmsh.model.addPhysicalGroup(3, [bound_box], tag=10) # Material ID: 6
gmsh.model.setPhysicalName(3, 10, "gap")            # Region name: "gap"

# Create a MathEval field to control the mesh size.
fieldTag = gmsh.model.mesh.field.add("MathEval") # ChatGPT -> "Fields in Gmsh are used to control the size, distribution, and behavior of the mesh"
gmsh.model.mesh.field.setString(fieldTag, "F", "1.0")


# Use this field as the background mesh field.
gmsh.model.mesh.field.setAsBackgroundMesh(fieldTag) # Mesh field applied to the entire geometry

# Generate the 3D mesh
gmsh.model.mesh.generate(3)

# Save mesh in VTK format
gmsh.write("msh_meshes/9_peb.msh")
gmsh.write("vtk_meshes/9_peb.vtk")

if "-nopopup" not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()



Volumes: [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10)]
Info    : Meshing 1D...
Info    : [ 10%] Meshing curve 14 (Circle)
Info    : [ 10%] Meshing curve 40 (Circle)
Info    : [ 10%] Meshing curve 42 (Circle)
Info    : [ 10%] Meshing curve 43 (Circle)
Info    : [ 10%] Meshing curve 44 (Line)
Info    : [ 20%] Meshing curve 45 (Line)
Info    : [ 20%] Meshing curve 46 (Line)
Info    : [ 20%] Meshing curve 59 (Circle)
Info    : [ 20%] Meshing curve 60 (Circle)
Info    : [ 20%] Meshing curve 61 (Circle)
Info    : [ 20%] Meshing curve 63 (Line)
Info    : [ 30%] Meshing curve 64 (Line)
Info    : [ 30%] Meshing curve 65 (Line)
Info    : [ 30%] Meshing curve 78 (Circle)
Info    : [ 30%] Meshing curve 79 (Circle)
Info    : [ 30%] Meshing curve 80 (Circle)
Info    : [ 30%] Meshing curve 82 (Line)
Info    : [ 40%] Meshing curve 83 (Line)
Info    : [ 40%] Meshing curve 84 (Line)
Info    : [ 40%] Meshing curve 97 (Circle)
Info    : [ 40%] Meshing curve 98 (Circle)
